## 기본 install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytesseract PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.8 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/STUBO/ksatparser")

In [ ]:
# Install Poppler utilities
!sudo apt-get update
!sudo apt-get install -y poppler-utils

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,267 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Get:13 http://security.ubuntu.com/ubun

In [ ]:
import base64
from PIL import Image
from IPython.display import display
from openai import OpenAI

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

## Easy OCR

In [ ]:
import os
import re
import json
import cv2
import numpy as np
import unicodedata
import easyocr
from openai import OpenAI
from PIL import Image

# ✅ 설정
image_dir = "/content/drive/MyDrive/classified_images/비문학"
wrong_log_path = "/content/wrong_answers.json"
client = OpenAI()
reader = easyocr.Reader(['ko', 'en'], gpu=True)

# ✅ 유니코드 정규화
def normalize_filename(fn):
    return unicodedata.normalize('NFC', fn)

# ✅ 정답지 (JSON에서 로드한 대신 코드에 직접 포함)
true_answers = {
    "2022-03": {1: "④", 2: "③", 3: "①", 4: "③", 5: "⑤", 6: "③", 7: "⑤", 8: "①", 9: "⑤", 10: "②", 11: "③", 12: "①", 13: "③", 14: "②", 15: "②", 16: "②", 17: "③"},
    "2022-06": {1: "⑤", 2: "①", 3: "③", 4: "③", 5: "④", 6: "④", 7: "②", 8: "②", 9: "①", 10: "③", 11: "⑤", 12: "④", 13: "②", 14: "①", 15: "②", 16: "④", 17: "②"},
    "2022-09": {1: "①", 2: "③", 3: "⑤", 4: "②", 5: "③", 6: "②", 7: "①", 8: "③", 9: "⑤", 10: "⑤", 11: "④", 12: "⑤", 13: "④", 14: "⑤", 15: "③", 16: "①", 17: "①"},
    "2022-수능": {1: "②", 2: "⑤", 3: "①", 4: "①", 5: "③", 6: "④", 7: "③", 8: "②", 9: "③", 10: "②", 11: "⑤", 12: "⑤", 13: "④", 14: "④", 15: "②", 16: "④", 17: "①"},
    "2023-03": {1: "③", 2: "⑤", 3: "④", 4: "④", 5: "②", 6: "⑤", 7: "④", 8: "①", 9: "④", 10: "①", 11: "②", 12: "⑤", 13: "①", 14: "②", 15: "③", 16: "⑤", 17: "②"},
    "2023-06": {1: "①", 2: "⑤", 3: "④", 4: "①", 5: "③", 6: "④", 7: "①", 8: "②", 9: "③", 10: "①", 11: "②", 12: "④", 13: "③", 14: "①", 15: "⑤", 16: "④", 17: "②"},
    "2023-09": {1: "①", 2: "④", 3: "①", 4: "③", 5: "①", 6: "⑤", 7: "⑤", 8: "③", 9: "①", 10: "②", 11: "④", 12: "②", 13: "④", 14: "②", 15: "⑤", 16: "⑤", 17: "①"},
    "2023-수능": {1: "④", 2: "⑤", 3: "①", 4: "④", 5: "⑤", 6: "③", 7: "②", 8: "⑤", 9: "②", 10: "④", 11: "⑤", 12: "②", 13: "⑤", 14: "③", 15: "④", 16: "④", 17: "①"},
    "2024-03": {1: "①", 2: "②", 3: "②", 4: "③", 5: "③", 6: "①", 7: "②", 8: "②", 9: "①", 10: "①", 11: "③", 12: "③", 13: "③", 14: "③", 15: "②", 16: "⑤", 17: "④"},
    "2024-06": {1: "②", 2: "⑤", 3: "①", 4: "②", 5: "④", 6: "⑤", 7: "⑤", 8: "②", 9: "①", 10: "④", 11: "③", 12: "①", 13: "③", 14: "①", 15: "②", 16: "③", 17: "④"},
    "2024-09": {1: "②", 2: "③", 3: "③", 4: "③", 5: "⑤", 6: "④", 7: "①", 8: "⑤", 9: "④", 10: "⑤", 11: "②", 12: "④", 13: "⑤", 14: "③", 15: "⑤", 16: "⑤", 17: "①"},
    "2024-수능": {1: "⑤", 2: "③", 3: "①", 4: "⑤", 5: "③", 6: "②", 7: "②", 8: "③", 9: "①", 10: "⑤", 11: "②", 12: "③", 13: "①", 14: "④", 15: "④", 16: "⑤", 17: "④"},
    "2025-03": {1: "②", 2: "④", 3: "④", 4: "②", 5: "③", 6: "③", 7: "②", 8: "⑤", 9: "①", 10: "⑤", 11: "④", 12: "⑤", 13: "③", 14: "②", 15: "①", 16: "①", 17: "⑤"},
    "2025-06": {1: "⑤", 2: "②", 3: "②", 4: "①", 5: "⑤", 6: "②", 7: "①", 8: "④", 9: "①", 10: "③", 11: "③", 12: "④", 13: "⑤", 14: "④", 15: "①", 16: "①", 17: "②"},
    "2025-09": {1: "④", 2: "⑤", 3: "②", 4: "④", 5: "④", 6: "③", 7: "①", 8: "④", 9: "⑤", 10: "②", 11: "③", 12: "①", 13: "③", 14: "⑤", 15: "①", 16: "⑤", 17: "①"},
    "2025-수능": {1: "③", 2: "④", 3: "⑤", 4: "④", 5: "⑤", 6: "③", 7: "②", 8: "①", 9: "②", 10: "③", 11: "①", 12: "⑤", 13: "③", 14: "①", 15: "②", 16: "②", 17: "③"},
}

# ✅ 텍스트 추출
def extract_text_with_underlines(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    results = reader.readtext(gray, detail=True)

    full_text = " ".join([text for (_, text, _) in results])
    return re.sub(r'\b1[\.\)]', '①', full_text)\
             .replace('2.', '②')\
             .replace('3.', '③')\
             .replace('4.', '④')\
             .replace('5.', '⑤')\
             .strip()


# ✅ GPT 프롬프트
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 질문 조건을 정확히 반영해 정답을 선택하세요.
2. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
3. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

아래 이미지는 문학 문제 하나의 '질문 문장 + ①~⑤ 선택지'가 포함된 이미지야.
    만약 <보기> 문장이 존재한다면 질문 앞에 위치하며, 반드시 포함해서 출력해.

    형식은 아래와 같이 출력해:

    (질문과 <보기> 내용. <보기>가 없다면 생략)
    ① ...
    ② ...
    ③ ...
    ④ ...
    ⑤ ...

    ❗ 절대 설명이나 부가 텍스트를 추가하지 말고 형식 그대로 출력해.

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)

'''

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 저장
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 매핑
def get_mapping(year, month):
    special = (year, month) in [(2024, "06"), (2024, "09"), (2024, "수능"), (2025, "06"), (2025, "09")]
    return {
        "p1": [1, 2, 3],
        "p2": list(range(4, 8)) if special else list(range(4, 10)),
        "p3": list(range(8, 12)) if special else list(range(10, 14)),
        "p4": list(range(12, 18)) if special else list(range(14, 18))
    }

# ✅ 실행 대상
target_sets = [
    #(2022, "03"), (2022, "06"), (2022, "09"), (2022, "수능"),
    #(2023, "03"), (2023, "06"), (2023, "09"), (2023, "수능"),
    #(2024, "03"), (2024, "06"), (2024, "09"), (2024, "수능"),
    #(2025, "03"), (2025, "06"), (2025, "09"),  (2025, "수능")
    (2025, "06"), (2025, "09"),  (2025, "수능")
]

wrong = 0

# ✅ 실행 루프
for year, month in target_sets:
    exam_key = f"{year}-{month}"
    if exam_key not in true_answers:
        print(f"⚠️ 정답 없음: {exam_key}")
        continue

    mapping = get_mapping(year, month)
    for p_key, q_nums in mapping.items():
        p_path = normalize_filename(f"{year}-{month}-국어_{p_key}.png")
        p_img = os.path.join(image_dir, p_path)
        if not os.path.exists(p_img):
            print(f"❌ 지문 이미지 없음: {p_img}")
            continue
        passage = extract_text_with_underlines(p_img)

        for qn in q_nums:
            q_path = normalize_filename(f"{year}-{month}-국어_{qn}.png")
            q_img = os.path.join(image_dir, q_path)
            if not os.path.exists(q_img):
                print(f"❌ 문제 이미지 없음: {q_img}")
                continue

            question = extract_text_with_underlines(q_img)
            prompt = text_prompt.format(passage=passage, question=question)
            print(f"\n📘 [{exam_key} / 문제 {qn}]")

            try:
                gpt_output = ask_gpt(prompt)
                gpt_ans, explanation = parse_gpt_output(gpt_output)
                true_ans = true_answers[exam_key].get(qn)

                print(f"GPT: {gpt_ans}, 정답: {true_ans}")
                if gpt_ans != true_ans:
                    print("❌ 오답 기록")
                    wrong += 1
                    qid = f"{exam_key}-{qn}"
                    log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
            except Exception as e:
                print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-06 / 문제 1]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 2]
GPT: ③, 정답: ②
❌ 오답 기록

📘 [2025-06 / 문제 3]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 4]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 5]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 6]
GPT: ③, 정답: ②
❌ 오답 기록

📘 [2025-06 / 문제 7]
GPT: ③, 정답: ①
❌ 오답 기록

📘 [2025-06 / 문제 8]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 9]
GPT: ②, 정답: ①
❌ 오답 기록

📘 [2025-06 / 문제 10]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 11]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 12]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 13]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 14]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 15]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 16]
GPT: ③, 정답: ①
❌ 오답 기록

📘 [2025-06 / 문제 17]
GPT: ③, 정답: ②
❌ 오답 기록

📘 [2025-09 / 문제 1]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 2]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 3]
GPT: ③, 정답: ②
❌ 오답 기록

📘 [2025-09 / 문제 4]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 5]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 6]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 7]
GPT: ④, 정답: ①
❌ 오답 기록

📘 [2025-09 / 문제 8]
GPT: ③, 정답: ④
❌ 오답 기록

📘 [2025-09 / 문제 9]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 10]
GPT: ④, 정답: ②
❌

## 정제된 텍스트 **(2025-06)**

In [ ]:
import os
import re
import json
from openai import OpenAI
import unicodedata

# ✅ 설정
text_path = "/content/2025-06.txt"
wrong_log_path = "/content/wrong_answers.json"
client = OpenAI()

true_answers = {
    "2025-06": {
        1: "⑤", 2: "②", 3: "②", 4: "①", 5: "⑤", 6: "②", 7: "①",
        8: "④", 9: "①", 10: "③", 11: "③", 12: "④", 13: "⑤", 14: "④",
        15: "①", 16: "①", 17: "②"
    }
}

# ✅ 프롬프트 템플릿 (강조, <보기>, [보기] 모두 유지)
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 실제 지문 내용과 선택지별 주장 및 결과가 정확히 일치하는지,
2. 반드시 각 선지별로 부합하면 ‘O’, 어긋나면 명확하게 ‘X’로 채점하고,
3. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
4. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

지문에는 **강조**된 단어, <구간> 문장, [보기] 문장이 포함되어 있을 수 있습니다.
아래 형식을 유지하세요:

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)
'''

# ✅ 텍스트 파일 파싱
def parse_text_by_blocks(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 지문 구간 분리 (예: [1~3], [4~7] ...)
    raw_blocks = re.split(r'\[(\d+～\d+)\]', content)
    passages = []
    i = 1
    while i < len(raw_blocks):
        range_str = raw_blocks[i]
        body = raw_blocks[i+1].strip()
        start, end = map(int, range_str.split('～'))
        questions = []

        # 문제 단락 분리 (<보기> 포함 가능)
        q_splits = re.split(r'\n\s*(\d{1,2})\.\s', body)
        q_texts = []
        j = 1
        while j < len(q_splits):
            q_num = int(q_splits[j])
            q_body = q_splits[j+1].strip()
            q_texts.append((q_num, q_body))
            j += 2

        passages.append({
            "range": (start, end),
            "text": q_splits[0].strip(),  # 지문 부분
            "questions": q_texts
        })
        i += 2

    return passages

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 기록
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 실행
exam_key = "2025-06"
wrong = 0
blocks = parse_text_by_blocks(text_path)

for block in blocks:
    passage = block["text"]
    for qn, question in block["questions"]:
        prompt = text_prompt.format(passage=passage, question=question)
        print(f"\n📘 [{exam_key} / 문제 {qn}]")
        try:
            gpt_output = ask_gpt(prompt)
            gpt_ans, explanation = parse_gpt_output(gpt_output)
            true_ans = true_answers[exam_key].get(qn)

            print(f"GPT: {gpt_ans}, 정답: {true_ans}")
            if gpt_ans != true_ans:
                print("❌ 오답 기록")
                wrong += 1
                qid = f"{exam_key}-{qn}"
                log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
        except Exception as e:
            print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-06 / 문제 1]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 2]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 3]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 4]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 5]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 6]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 7]
GPT: ③, 정답: ①
❌ 오답 기록

📘 [2025-06 / 문제 8]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 9]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 10]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 11]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 12]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 13]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 14]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 15]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 16]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 17]
GPT: ②, 정답: ②

✅ 오답 개수: 1


힌트추가 (7번)

In [ ]:
import os
import re
import json
from openai import OpenAI
import unicodedata

# ✅ 설정
text_path = "/content/2025-06.txt"
wrong_log_path = "/content/wrong_answers.json"
client = OpenAI()

true_answers = {
    "2025-06": {
        1: "⑤", 2: "②", 3: "②", 4: "①", 5: "⑤", 6: "②", 7: "①",
        8: "④", 9: "①", 10: "③", 11: "③", 12: "④", 13: "⑤", 14: "④",
        15: "①", 16: "①", 17: "②"
    }
}

# ✅ 프롬프트 템플릿 (강조, <보기>, [보기] 모두 유지)
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 실제 지문 내용과 선택지별 주장 및 결과가 정확히 일치하는지,
2. 반드시 각 선지별로 부합하면 ‘O’, 어긋나면 명확하게 ‘X’로 채점하고,
3. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
4. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

지문에는 **강조**된 단어, <구간> 문장, [보기] 문장이 포함되어 있을 수 있습니다.
아래 형식을 유지하세요:

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)
'''

# ✅ 텍스트 파일 파싱
def parse_text_by_blocks(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 지문 구간 분리 (예: [1~3], [4~7] ...)
    raw_blocks = re.split(r'\[(\d+～\d+)\]', content)
    passages = []
    i = 1
    while i < len(raw_blocks):
        range_str = raw_blocks[i]
        body = raw_blocks[i+1].strip()
        start, end = map(int, range_str.split('～'))
        questions = []

        # 문제 단락 분리 (<보기> 포함 가능)
        q_splits = re.split(r'\n\s*(\d{1,2})\.\s', body)
        q_texts = []
        j = 1
        while j < len(q_splits):
            q_num = int(q_splits[j])
            q_body = q_splits[j+1].strip()
            q_texts.append((q_num, q_body))
            j += 2

        passages.append({
            "range": (start, end),
            "text": q_splits[0].strip(),  # 지문 부분
            "questions": q_texts
        })
        i += 2

    return passages

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 기록
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 실행
exam_key = "2025-06"
wrong = 0
blocks = parse_text_by_blocks(text_path)

for block in blocks:
    passage = block["text"]
    for qn, question in block["questions"]:
        prompt = text_prompt.format(passage=passage, question=question)
        print(f"\n📘 [{exam_key} / 문제 {qn}]")
        try:
            gpt_output = ask_gpt(prompt)
            gpt_ans, explanation = parse_gpt_output(gpt_output)
            true_ans = true_answers[exam_key].get(qn)

            print(f"GPT: {gpt_ans}, 정답: {true_ans}")
            if gpt_ans != true_ans:
                print("❌ 오답 기록")
                wrong += 1
                qid = f"{exam_key}-{qn}"
                log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
        except Exception as e:
            print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-06 / 문제 1]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 2]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 3]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 4]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 5]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 6]
GPT: ②, 정답: ②

📘 [2025-06 / 문제 7]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 8]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 9]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 10]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 11]
GPT: ③, 정답: ③

📘 [2025-06 / 문제 12]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 13]
GPT: ⑤, 정답: ⑤

📘 [2025-06 / 문제 14]
GPT: ④, 정답: ④

📘 [2025-06 / 문제 15]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 16]
GPT: ①, 정답: ①

📘 [2025-06 / 문제 17]
GPT: ②, 정답: ②

✅ 오답 개수: 0


## 정제된 텍스트 **(2025-09)**

In [ ]:
import os
import re
import json
from openai import OpenAI
import unicodedata

# ✅ 설정
text_path = "/content/2025-09.txt"
wrong_log_path = "/content/wrong_answers.json"
client = OpenAI()

true_answers = {
    "2025-09": {1: "④", 2: "⑤", 3: "②", 4: "④", 5: "④", 6: "③", 7: "①", 8: "④", 9: "⑤", 10: "②", 11: "③", 12: "①", 13: "③", 14: "⑤", 15: "①", 16: "⑤", 17: "①"},

}

# ✅ 프롬프트 템플릿 (강조, <보기>, [보기] 모두 유지)
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 실제 지문 내용과 선택지별 주장 및 결과가 정확히 일치하는지,
2. 반드시 각 선지별로 부합하면 ‘O’, 어긋나면 명확하게 ‘X’로 채점하고,
3. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
4. 선택지 중 포괄관계, 관계역전 오류를 주의하세요.
5. 어휘 문맥 판단 문제에서는 사전적 의미를 판단해서 답변하세요.
4. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

지문에는 **강조**된 단어, <구간> 문장, [보기] 문장이 포함되어 있을 수 있습니다.
아래 형식을 유지하세요:

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)
'''

# ✅ 텍스트 파일 파싱
def parse_text_by_blocks(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 지문 구간 분리 (예: [1~3], [4~7] ...)
    raw_blocks = re.split(r'\[(\d+～\d+)\]', content)
    passages = []
    i = 1
    while i < len(raw_blocks):
        range_str = raw_blocks[i]
        body = raw_blocks[i+1].strip()
        start, end = map(int, range_str.split('～'))
        questions = []

        # 문제 단락 분리 (<보기> 포함 가능)
        q_splits = re.split(r'\n\s*(\d{1,2})\.\s', body)
        q_texts = []
        j = 1
        while j < len(q_splits):
            q_num = int(q_splits[j])
            q_body = q_splits[j+1].strip()
            q_texts.append((q_num, q_body))
            j += 2

        passages.append({
            "range": (start, end),
            "text": q_splits[0].strip(),  # 지문 부분
            "questions": q_texts
        })
        i += 2

    return passages

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 기록
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 실행
exam_key = "2025-09"
wrong = 0
blocks = parse_text_by_blocks(text_path)

for block in blocks:
    passage = block["text"]
    for qn, question in block["questions"]:
        prompt = text_prompt.format(passage=passage, question=question)
        print(f"\n📘 [{exam_key} / 문제 {qn}]")
        try:
            gpt_output = ask_gpt(prompt)
            gpt_ans, explanation = parse_gpt_output(gpt_output)
            true_ans = true_answers[exam_key].get(qn)

            print(f"GPT: {gpt_ans}, 정답: {true_ans}")
            if gpt_ans != true_ans:
                print("❌ 오답 기록")
                wrong += 1
                qid = f"{exam_key}-{qn}"
                log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
        except Exception as e:
            print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-09 / 문제 1]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 2]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 3]
GPT: ②, 정답: ②

📘 [2025-09 / 문제 4]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 5]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 6]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 7]
GPT: ④, 정답: ①
❌ 오답 기록

📘 [2025-09 / 문제 8]
GPT: ③, 정답: ④
❌ 오답 기록

📘 [2025-09 / 문제 9]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 10]
GPT: ②, 정답: ②

📘 [2025-09 / 문제 11]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 12]
GPT: ①, 정답: ①

📘 [2025-09 / 문제 13]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 14]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 15]
GPT: ①, 정답: ①

📘 [2025-09 / 문제 16]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 17]
GPT: ①, 정답: ①

✅ 오답 개수: 2


힌트 추가

In [ ]:
import os
import re
import json
from openai import OpenAI
import unicodedata

# ✅ 설정
text_path = "/content/2025-09.txt"
wrong_log_path = "/content/wrong_answers.json"
client = OpenAI()

true_answers = {
    "2025-09": {1: "④", 2: "⑤", 3: "②", 4: "④", 5: "④", 6: "③", 7: "①", 8: "④", 9: "⑤", 10: "②", 11: "③", 12: "①", 13: "③", 14: "⑤", 15: "①", 16: "⑤", 17: "①"},

}

# ✅ 프롬프트 템플릿 (강조, <보기>, [보기] 모두 유지)
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 실제 지문 내용과 선택지별 주장 및 결과가 정확히 일치하는지,
2. 반드시 각 선지별로 부합하면 ‘O’, 어긋나면 명확하게 ‘X’로 채점하고,
3. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
4. 선택지 중 포괄관계, 관계역전 오류를 주의하세요.
5. 어휘 문맥 판단 문제에서는 사전적 의미를 판단해서 답변하세요.
4. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

지문에는 **강조**된 단어, <구간> 문장, [보기] 문장이 포함되어 있을 수 있습니다.
아래 형식을 유지하세요:

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)
'''

# ✅ 텍스트 파일 파싱
def parse_text_by_blocks(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 지문 구간 분리 (예: [1~3], [4~7] ...)
    raw_blocks = re.split(r'\[(\d+～\d+)\]', content)
    passages = []
    i = 1
    while i < len(raw_blocks):
        range_str = raw_blocks[i]
        body = raw_blocks[i+1].strip()
        start, end = map(int, range_str.split('～'))
        questions = []

        # 문제 단락 분리 (<보기> 포함 가능)
        q_splits = re.split(r'\n\s*(\d{1,2})\.\s', body)
        q_texts = []
        j = 1
        while j < len(q_splits):
            q_num = int(q_splits[j])
            q_body = q_splits[j+1].strip()
            q_texts.append((q_num, q_body))
            j += 2

        passages.append({
            "range": (start, end),
            "text": q_splits[0].strip(),  # 지문 부분
            "questions": q_texts
        })
        i += 2

    return passages

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 기록
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 실행
exam_key = "2025-09"
wrong = 0
blocks = parse_text_by_blocks(text_path)

for block in blocks:
    passage = block["text"]
    for qn, question in block["questions"]:
        prompt = text_prompt.format(passage=passage, question=question)
        print(f"\n📘 [{exam_key} / 문제 {qn}]")
        try:
            gpt_output = ask_gpt(prompt)
            gpt_ans, explanation = parse_gpt_output(gpt_output)
            true_ans = true_answers[exam_key].get(qn)

            print(f"GPT: {gpt_ans}, 정답: {true_ans}")
            if gpt_ans != true_ans:
                print("❌ 오답 기록")
                wrong += 1
                qid = f"{exam_key}-{qn}"
                log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
        except Exception as e:
            print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-09 / 문제 1]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 2]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 3]
GPT: ②, 정답: ②

📘 [2025-09 / 문제 4]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 5]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 6]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 7]
GPT: ①, 정답: ①

📘 [2025-09 / 문제 8]
GPT: ④, 정답: ④

📘 [2025-09 / 문제 9]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 10]
GPT: ②, 정답: ②

📘 [2025-09 / 문제 11]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 12]
GPT: ①, 정답: ①

📘 [2025-09 / 문제 13]
GPT: ③, 정답: ③

📘 [2025-09 / 문제 14]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 15]
GPT: ①, 정답: ①

📘 [2025-09 / 문제 16]
GPT: ⑤, 정답: ⑤

📘 [2025-09 / 문제 17]
GPT: ①, 정답: ①

✅ 오답 개수: 0


## 정제된 텍스트 **(2025-수능)**

In [ ]:
import os
import re
import json
from openai import OpenAI
import unicodedata

# ✅ 설정
text_path = "/content/2025-수능.txt"
wrong_log_path = "/content/wrong_answers(2025-수능).json"
client = OpenAI()

true_answers = {
    "2025-수능": {1: "③", 2: "④", 3: "⑤", 4: "④", 5: "⑤", 6: "③", 7: "②", 8: "①", 9: "②", 10: "③", 11: "①", 12: "⑤", 13: "③", 14: "①", 15: "②", 16: "②", 17: "③"},
}

# ✅ 프롬프트 템플릿 (강조, <보기>, [보기] 모두 유지)
text_prompt = '''
다음은 국어 비문학 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 실제 지문 내용과 선택지별 주장 및 결과가 정확히 일치하는지,
2. 반드시 각 선지별로 부합하면 ‘O’, 어긋나면 명확하게 ‘X’로 채점하고,
3. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
4. 선택지 중 포괄관계, 관계역전 오류를 주의하세요.
5. 어휘 문맥 판단 문제에서는 사전적 의미를 판단해서 답변하세요.
4. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

지문에는 **강조**된 단어, <구간> 문장, [보기] 문장이 포함되어 있을 수 있습니다.
아래 형식을 유지하세요:

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)
'''

# ✅ 텍스트 파일 파싱
def parse_text_by_blocks(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 지문 구간 분리 (예: [1~3], [4~7] ...)
    raw_blocks = re.split(r'\[(\d+～\d+)\]', content)
    passages = []
    i = 1
    while i < len(raw_blocks):
        range_str = raw_blocks[i]
        body = raw_blocks[i+1].strip()
        start, end = map(int, range_str.split('～'))
        questions = []

        # 문제 단락 분리 (<보기> 포함 가능)
        q_splits = re.split(r'\n\s*(\d{1,2})\.\s', body)
        q_texts = []
        j = 1
        while j < len(q_splits):
            q_num = int(q_splits[j])
            q_body = q_splits[j+1].strip()
            q_texts.append((q_num, q_body))
            j += 2

        passages.append({
            "range": (start, end),
            "text": q_splits[0].strip(),  # 지문 부분
            "questions": q_texts
        })
        i += 2

    return passages

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

# ✅ 오답 기록
def log_wrong_answer(qid, gpt_answer, true_answer, passage, question, explanation):
    entry = {
        "문제ID": qid,
        "GPT_정답": gpt_answer,
        "실제정답": true_answer,
        "지문": passage,
        "문제": question,
        "GPT_해설": explanation
    }
    if os.path.exists(wrong_log_path):
        with open(wrong_log_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []
    data.append(entry)
    with open(wrong_log_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 실행
exam_key = "2025-수능"
wrong = 0
blocks = parse_text_by_blocks(text_path)

for block in blocks:
    passage = block["text"]
    for qn, question in block["questions"]:
        prompt = text_prompt.format(passage=passage, question=question)
        print(f"\n📘 [{exam_key} / 문제 {qn}]")
        try:
            gpt_output = ask_gpt(prompt)
            gpt_ans, explanation = parse_gpt_output(gpt_output)
            true_ans = true_answers[exam_key].get(qn)

            print(f"GPT: {gpt_ans}, 정답: {true_ans}")
            if gpt_ans != true_ans:
                print("❌ 오답 기록")
                wrong += 1
                qid = f"{exam_key}-{qn}"
                log_wrong_answer(qid, gpt_ans, true_ans, passage, question, explanation)
        except Exception as e:
            print(f"⚠️ GPT 실패: {e}")

print(f"\n✅ 오답 개수: {wrong}")



📘 [2025-수능 / 문제 1]
GPT: ③, 정답: ③

📘 [2025-수능 / 문제 2]
GPT: ④, 정답: ④

📘 [2025-수능 / 문제 3]
GPT: ⑤, 정답: ⑤

📘 [2025-수능 / 문제 4]
GPT: ④, 정답: ④

📘 [2025-수능 / 문제 5]
GPT: ⑤, 정답: ⑤

📘 [2025-수능 / 문제 6]
GPT: ③, 정답: ③

📘 [2025-수능 / 문제 7]
GPT: ③, 정답: ②
❌ 오답 기록

📘 [2025-수능 / 문제 8]
GPT: ⑤, 정답: ①
❌ 오답 기록

📘 [2025-수능 / 문제 9]
GPT: ②, 정답: ②

📘 [2025-수능 / 문제 10]
GPT: ③, 정답: ③

📘 [2025-수능 / 문제 11]
GPT: ①, 정답: ①

📘 [2025-수능 / 문제 12]
GPT: ⑤, 정답: ⑤

📘 [2025-수능 / 문제 13]
GPT: ③, 정답: ③

📘 [2025-수능 / 문제 14]
GPT: ①, 정답: ①

📘 [2025-수능 / 문제 15]
GPT: ②, 정답: ②

📘 [2025-수능 / 문제 16]
GPT: ④, 정답: ②
❌ 오답 기록

📘 [2025-수능 / 문제 17]
GPT: ③, 정답: ③

✅ 오답 개수: 3
